In [2]:
import pandas as pd

from ssrq_retro_lab.config import PROJECT_ROOT, ZG_DATA_ROOT
from ssrq_retro_lab.pipeline.templates.utils import render_template
from ssrq_retro_lab.repository.writer import JSONLWriter

/var/folders/zz/66rgnps56_7bx2sjt1f7rsh80000gn/T/ipykernel_54194/3814869860.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
ocr_training_df = pd.read_pickle(
    PROJECT_ROOT / "notebooks" / "pkl_cache" / "ocr_line_based_training.pkl"
)

In [4]:
ocr_training_df = pd.read_pickle(
    PROJECT_ROOT / "notebooks" / "pkl_cache" / "ocr_line_based_training.pkl"
)

In [5]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(ocr_training_df, test_size=0.2, random_state=42)

In [6]:
len(train), len(test)

(493, 124)

In [19]:
import json

from ssrq_retro_lab.train.messages import SYSTEM_ROLE_V2
from ssrq_retro_lab.pipeline.components.ocr_corrector import CorrectedOCRText

schema = json.dumps(CorrectedOCRText.model_json_schema(), indent=2)

In [58]:
def create_training_messages(batches: list[pd.DataFrame]) -> list[dict[str, list[dict[str, str]]]]:
    training_messages = []
    for batch in batches:
        messages = {"messages": []}

        user_input = render_template(
            template_name="openai_ocr_training_user_v2.jinja2",
            schema=schema,
            text_input=json.dumps(batch["source"].tolist(), ensure_ascii=False),
        )
        assistant_output = (
            "```json\n"
            + CorrectedOCRText(text=batch["target"].tolist()).model_dump_json(indent=2)
            + "\n```"
        )

        messages["messages"].append(
            {
                "role": "system",
                "content": SYSTEM_ROLE_V2,
            }
        )
        messages["messages"].append(
            {
                "role": "user",
                "content": user_input,
            }
        )
        messages["messages"].append(
            {
                "role": "assistant",
                "content": assistant_output,
            }
        )
        training_messages.append(messages)
    return training_messages

In [59]:
import numpy as np

training_batches: list[pd.DataFrame] = [
    batch for _, batch in train.groupby(np.arange(len(train)) // 5)
]

validation_batches: list[pd.DataFrame] = [
    batch for _, batch in test.groupby(np.arange(len(test)) // 5)
]

training_messages: list[dict[str, list[dict[str, str]]]] = create_training_messages(
    training_batches
)

validation_messages: list[dict[str, list[dict[str, str]]]] = create_training_messages(
    validation_batches
)

In [60]:
JSONLWriter(ZG_DATA_ROOT / "training_data" / "gpt_ocr" / "train.jsonl").write(
    content=training_messages
)

JSONLWriter(ZG_DATA_ROOT / "training_data" / "gpt_ocr" / "test.jsonl").write(
    content=training_messages
)

In [61]:
from ssrq_retro_lab.train.stats import (
    num_assistant_tokens_from_messages,
    num_tokens_from_messages,
    print_distribution,
)

TOKEN_LIMIT = 16385

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in training_messages:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > TOKEN_LIMIT for l in convo_lens)
print(
    f"\n{n_too_long} examples may be over the {TOKEN_LIMIT} token limit, they will be truncated during fine-tuning"
)

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 409, 606
mean / median: 509.42424242424244, 509.0
p5 / p95: 471.0, 555.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 76, 172
mean / median: 126.52525252525253, 127.0
p5 / p95: 107.6, 150.0

0 examples may be over the 16385 token limit, they will be truncated during fine-tuning


In [62]:
# Pricing and default n_epochs estimate

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 75
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(messages)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(TOKEN_LIMIT, length) for length in convo_lens)
print(
    f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training"
)
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(
    f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens"
)

cost_per_100k_tokens = 0.80  # Cost for every 100,000 tokens
estimated_cost = (
    (n_epochs * n_billing_tokens_in_dataset) / 100000
) * cost_per_100k_tokens
print(
    f"Estimated cost for fine-tuning: approximately ${estimated_cost:.2f}"
)  # I added this for actual cost based on current pricing

Dataset has ~50433 tokens that will be charged for during training
By default, you'll train for 25 epochs on this dataset
By default, you'll be charged for ~1260825 tokens
Estimated cost for fine-tuning: approximately $10.09


In [70]:
from ssrq_retro_lab.train import train

In [71]:
training_file_id = train.upload_training_file_to_openai(
    (ZG_DATA_ROOT / "training_data" / "gpt_ocr" / "train.jsonl")
)

test_file_id = train.upload_training_file_to_openai(
    (ZG_DATA_ROOT / "training_data" / "gpt_ocr" / "test.jsonl")
)

In [74]:
import openai

job_response = openai.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=test_file_id,
    model="gpt-3.5-turbo-1106",
    suffix="ssrq-ocr-cor",
    hyperparameters={"n_epochs": 4},
)

In [80]:
finetuned_model_id = model_id = train.get_finetuned_model_id(
    job_response.id
)

finetuned_model_id

'ft:gpt-3.5-turbo-1106:personal:ssrq-ocr-cor:8tgnqalq'

In [7]:
test.to_pickle("./pkl_cache/openai_finetune_ocr_validation_v2.pkl")